https://pythonprogramming.net/sklearn-scikit-learn-nltk-tutorial/

### Combine NLTK with Sklearn

to reap the benefits of Sklearn to use for NLP

In [1]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
import pickle

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode


In [2]:
# 2000 movie reviews: 1000 negative, 1000 positive
# each element in documents is a tuple consisting of the tokenized review and the label
documents = [(list(movie_reviews.words(fileid)), category)
            for category in movie_reviews.categories()
            for fileid in movie_reviews.fileids(category)]


In [3]:
# shuffle the data
random.shuffle(documents)


In [4]:
# get all words in the movie reviews and convert ot lower case
all_words = []

for w in movie_reviews.words():
    all_words.append(w.lower())
    

In [5]:
# frequency distribution of words
all_words = nltk.FreqDist(all_words)


#### Convert words into features

In [6]:
# limit number of words (only 3,000 most common words)
word_features = list(all_words.keys())[:3000]


In [7]:
# Find the most common words (if they are present in a movie review)
def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
        
    return features


In [8]:
featuresets = [(find_features(rev), category) for (rev,category) in documents]

# 2000 movie reviews
len(featuresets)


2000

#### Test different classifiers

In [9]:
# Train and test sets
training_set = featuresets[:1900]
testing_set = featuresets[1900:]


In [10]:
# Naive Bayes Classifier
classifier = nltk.NaiveBayesClassifier.train(training_set)

# Test the classifier
print('Original NB Classifier accuracy:', nltk.classify.accuracy(classifier, testing_set))


Original NB Classifier accuracy: 0.83


In [11]:
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)

print('MNB_classifier accuracy:', nltk.classify.accuracy(MNB_classifier, testing_set))


MNB_classifier accuracy: 0.84


In [12]:
# Classify one test example
print(MNB_classifier.classify_many(testing_set[0][0]))
print(MNB_classifier.classify(testing_set[0][0]))


['pos']
pos


In [13]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)

print('BernoulliNB_classifier accuracy:', nltk.classify.accuracy(BernoulliNB_classifier, testing_set))


BernoulliNB_classifier accuracy: 0.83


In [14]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)

print('LogisticRegression_classifier accuracy:', (nltk.classify.accuracy(LogisticRegression_classifier, testing_set)))


C:\Users\Christoph\anaconda3\envs\NLP\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression_classifier accuracy: 0.84


In [15]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)

print('SGDClassifier_classifier accuracy:', (nltk.classify.accuracy(SGDClassifier_classifier, testing_set)))


SGDClassifier_classifier accuracy: 0.84


In [16]:
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)

print('SVC_classifier accuracy:', (nltk.classify.accuracy(SVC_classifier, testing_set)))


SVC_classifier accuracy: 0.85


In [17]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)

print('LinearSVC_classifier accuracy:', (nltk.classify.accuracy(LinearSVC_classifier, testing_set)))


LinearSVC_classifier accuracy: 0.81


In [18]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)

print('NuSVC_classifier accuracy:', (nltk.classify.accuracy(NuSVC_classifier, testing_set)))


NuSVC_classifier accuracy: 0.88


#### Combine algorithms for a voting system

https://pythonprogramming.net/combine-classifier-algorithms-nltk-tutorial/

it is best to have an odd number of classifiers, e.g., 4/7 vs. 3/7

In [19]:
class VoteClassifier(ClassifierI):
    # init method will always run in a class
    def __init__(self, *classifiers):
        self._classifiers = classifiers
        
    # classify method
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
    

In [20]:
voted_classifier = VoteClassifier(classifier,
                                  NuSVC_classifier,
                                  LinearSVC_classifier,
                                  SGDClassifier_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print('voted_classifier accuracy:', (nltk.classify.accuracy(voted_classifier, testing_set)))


voted_classifier accuracy: 0.85


In [21]:
print('Classification:', voted_classifier.classify(testing_set[0][0]), 'Confidence:',voted_classifier.confidence(testing_set[0][0]))
print('Classification:', voted_classifier.classify(testing_set[1][0]), 'Confidence:',voted_classifier.confidence(testing_set[1][0]))
print('Classification:', voted_classifier.classify(testing_set[2][0]), 'Confidence:',voted_classifier.confidence(testing_set[2][0]))
print('Classification:', voted_classifier.classify(testing_set[3][0]), 'Confidence:',voted_classifier.confidence(testing_set[3][0]))
print('Classification:', voted_classifier.classify(testing_set[4][0]), 'Confidence:',voted_classifier.confidence(testing_set[4][0]))
print('Classification:', voted_classifier.classify(testing_set[5][0]), 'Confidence:',voted_classifier.confidence(testing_set[5][0]))


Classification: pos Confidence: 1.0
Classification: neg Confidence: 1.0
Classification: pos Confidence: 1.0
Classification: neg Confidence: 1.0
Classification: neg Confidence: 1.0
Classification: pos Confidence: 0.7142857142857143
